In [1]:
import pandas as pd
import numpy as np

In [2]:
path='../Result_trip/'
ctrl1=pd.read_csv(path+'ctrl1_fitting.csv',low_memory=False)
ctrl2=pd.read_csv(path+'ctrl2_fitting.csv',low_memory=False)
ctrl3=pd.read_csv(path+'ctrl3_fitting.csv',low_memory=False)
treated1=pd.read_csv(path+'treated1_fitting.csv',low_memory=False)
treated2=pd.read_csv(path+'treated2_fitting.csv',low_memory=False)
treated3=pd.read_csv(path+'treated3_fitting.csv',low_memory=False)
ctrl1.loc[ctrl1[(ctrl1['ctrl1_R']<0.8) | (ctrl1['ctrl1_Pl']>0.3)].index,ctrl1.columns[4:]]=float('nan')
ctrl2.loc[ctrl2[(ctrl2['ctrl2_R']<0.8) | (ctrl2['ctrl2_Pl']>0.3)].index,ctrl2.columns[4:]]=float('nan')
ctrl3.loc[ctrl3[(ctrl3['ctrl3_R']<0.8) | (ctrl3['ctrl3_Pl']>0.3)].index,ctrl3.columns[4:]]=float('nan')
treated1.loc[treated1[treated1['treated1_R']<0.8].index,treated1.columns[4:]]=float('nan')
treated2.loc[treated2[treated2['treated2_R']<0.8].index,treated2.columns[4:]]=float('nan')
treated3.loc[treated3[treated3['treated3_R']<0.8].index,treated3.columns[4:]]=float('nan')

In [14]:
diff_rep1 = pd.DataFrame(index=range(0,ctrl1.shape[0]))
diff_rep1['diff_rep1'] = treated1['treated1_min']-ctrl1['ctrl1_min']
diff_rep1['slope_rep1'] = np.fmin(ctrl1['ctrl1_slope'],treated1['treated1_slope'])
diff_rep1=pd.concat([ctrl1[['Sequence','Protein_Group_Accessions']],diff_rep1],1)
diff_rep2 = pd.DataFrame(index=range(0,ctrl2.shape[0]))
diff_rep2['diff_rep2'] = treated2['treated2_min']-ctrl2['ctrl2_min']
diff_rep2['slope_rep2'] = np.fmin(ctrl2['ctrl2_slope'],treated2['treated2_slope'])
diff_rep2=pd.concat([ctrl2[['Sequence','Protein_Group_Accessions']],diff_rep2],1)
diff_rep3 = pd.DataFrame(index=range(0,ctrl3.shape[0]))
diff_rep3['diff_rep3'] = treated3['treated3_min']-ctrl3['ctrl3_min']
diff_rep3['slope_rep3'] = np.minimum(ctrl3['ctrl3_slope'],treated3['treated3_slope'])
diff_rep3=pd.concat([ctrl3[['Sequence','Protein_Group_Accessions']],diff_rep3],1)

In [15]:
def combine_col(df,string):
    df1 = pd.DataFrame(index=range(0,df.shape[0]))
    df1['mp_'+string]=df[string+'_min']
    df1=pd.concat([df[df.columns[1:3]],df1,df[df.columns[3:13]]],1)
    return df1

In [16]:
ctrl1fit=combine_col(ctrl1,'ctrl1')
ctrl2fit=combine_col(ctrl2,'ctrl2')
ctrl3fit=combine_col(ctrl3,'ctrl3')
treated1fit=combine_col(treated1,'treated1')
treated2fit=combine_col(treated2,'treated2')
treated3fit=combine_col(treated3,'treated3')

In [17]:
result=ctrl1fit.merge(treated1fit,on=["Sequence", "Protein_Group_Accessions"]).merge(ctrl2fit,on=["Sequence", "Protein_Group_Accessions"])\
.merge(treated2fit,on=["Sequence", "Protein_Group_Accessions"]).merge(ctrl3fit,on=["Sequence", "Protein_Group_Accessions"])\
.merge(treated3fit,on=["Sequence", "Protein_Group_Accessions"]).merge(diff_rep1,on=["Sequence", "Protein_Group_Accessions"])\
.merge(diff_rep2,on=["Sequence", "Protein_Group_Accessions"]).merge(diff_rep3,on=["Sequence", "Protein_Group_Accessions"])

result.to_csv(path+'peptides_fit.csv',index=False)